In [44]:
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-twitter-25')
glove_vectors.most_similar('capitalism')

[('socialism', 0.9675610661506653),
 ('tyranny', 0.9390194416046143),
 ('inequality', 0.9126962423324585),
 ('oppression', 0.9065089225769043),
 ('greed', 0.9040176868438721),
 ('fascism', 0.8969089388847351),
 ('capitalist', 0.8934274911880493),
 ('democracy', 0.8926239013671875),
 ('morality', 0.8892533183097839),
 ('poverty', 0.877461314201355)]

In [42]:
from xml.dom import minidom
import psycopg2
conn = psycopg2.connect(
    host="localhost",
    database="datamining",
    user="msbean",
    password="cactusgreen")

conn.autocommit = True
cursor = conn.cursor() 

for v in range(50):
    dom = minidom.parse("../videos/" + str(v) + "_captions.txt")
    nones = 0
    somes = 0
    for caption in dom.getElementsByTagName("s"):
        for child_node in caption.childNodes:
            if child_node.nodeType == child_node.TEXT_NODE:
                stripped = ''.join([i for i in child_node.data if i.isalpha()]).lower()
                if stripped in glove_vectors:
                    embedding = glove_vectors[stripped]
                    
                    sql = "INSERT INTO caption_embeddings (video_path, word, embedding) VALUES ('"
                    sql += "../videos/" + str(v) + ".mp4', '"
                    sql += stripped
                    sql += "', "
                    sql += "'["
                    for datum in embedding:
                        sql += str(datum) + ","
                    sql = sql[:-1]
                    sql += "]')"

                    cursor.execute(sql) 
                    somes += 1
                else:
                    nones += 1
    print(somes / (somes + nones))

conn.commit() 
conn.close() 

0.9787234042553191
0.9650735294117647
0.9765625
0.9694656488549618
0.9800399201596807
0.9743589743589743
0.9776876267748479
0.9757009345794393
0.98014440433213
0.9855855855855856
0.9769673704414588
0.9631675874769797
0.9658634538152611
0.9771428571428571
0.9823874755381604
0.974304068522484
0.9758551307847082
0.9873949579831933
0.9701789264413518
0.9733333333333334
0.9709618874773139
0.985239852398524
0.9884393063583815
0.972457627118644
0.9874776386404294
0.9956896551724138
0.9948717948717949
0.9946666666666667
1.0
0.9970760233918129
0.9976635514018691
0.9732510288065843
0.9975247524752475
1.0
0.9688715953307393
0.9792843691148776
0.9757462686567164
0.9723865877712031
0.9805068226120858
0.9827856025039123
0.9671179883945842
0.9746588693957114
0.9796557120500783
0.979253112033195
0.9689922480620154
0.9973404255319149
0.9736842105263158
0.9731543624161074
0.9865384615384616
0.9916142557651991
